In [201]:
import pandas as pd
import os
import numpy as np
import sys
sys.path.append('..')

In [203]:
#os.chdir(r'C:\Users\Administrator\Desktop\raphi_other\repositories\template_project\src')

In [204]:
data = pd.read_csv(r'../data/raw/data/scout_data/Case_Study_Data.csv', delimiter=';')

In [205]:
#from eda_first import reduce_mem_usage
#data = reduce_mem_usage(data)

In [207]:
data.columns

Index(['article_id', 'product_tier', 'make_name', 'price', 'first_zip_digit',
       'first_registration_year', 'created_date', 'deleted_date',
       'search_views', 'detail_views', 'stock_days', 'ctr'],
      dtype='object')

In [210]:
data.shape

(78321, 12)

In [212]:
data.head()

,article_id,product_tier,make_name,price,first_zip_digit,first_registration_year,created_date,deleted_date,search_views,detail_views,stock_days,ctr
0,350625839,Basic,Mitsubishi,16750,5,2013,24.07.18,24.08.18,3091.0000,123.0000,30,0.03780329990294403
1,354412280,Basic,Mercedes-Benz,35950,4,2015,16.08.18,07.10.18,3283.0000,223.0000,52,0.06792567773378008
2,349572992,Basic,Mercedes-Benz,11950,3,1998,16.07.18,05.09.18,3247.0000,265.0000,51,0.0816137973514013
3,350266763,Basic,Ford,1750,6,2003,20.07.18,29.10.18,1856.0000,26.0000,101,0.014008620689655173
4,355688985,Basic,Mercedes-Benz,26500,3,2014,28.08.18,08.09.18,490.0000,20.0000,12,0.04081632653061224


In [215]:
pd.set_option('display.float_format', '{:.4f}'.format)

In [217]:
data.describe()

,article_id,price,first_zip_digit,first_registration_year,search_views,detail_views,stock_days
count,78321.0000,78321.0000,78321.0000,78321.0000,78311.0000,78311.0000,78321.0000
mean,357486886.4943,15067.6553,4.6321,2011.0895,2297.5025,93.4699,35.9863
std,5076572.0239,16374.4976,2.3543,6.5384,6339.0344,228.0256,32.2148
min,347232386.0000,1.0000,1.0000,1924.0000,0.0000,0.0000,-3.0000
25%,353638688.0000,5750.0000,3.0000,2008.0000,367.0000,13.0000,10.0000
50%,358547952.0000,10900.0000,5.0000,2013.0000,920.0000,36.0000,25.0000
75%,361481657.0000,18890.0000,7.0000,2015.0000,2234.0000,94.0000,55.0000
max,364703971.0000,249888.0000,9.0000,2106.0000,608754.0000,13926.0000,127.0000


In [219]:
pct = np.percentile(data.loc[:, 'search_views'].fillna(np.mean(data.loc[:, 'search_views'])), 95)
print(pct)
print(data.shape)
data = data.loc[data.loc[:, 'search_views'] < pct]
print(data.shape)

7915.0
(78321, 12)
(74394, 12)


In [221]:
pct = np.percentile(data.loc[:, 'detail_views'].fillna(np.mean(data.loc[:, 'detail_views'])), 95)
print(pct)
print(data.shape)
data = data.loc[data.loc[:, 'detail_views'] < pct]
print(data.shape)

224.0
(74394, 12)
(70624, 12)


# DROP COLUMNS WHICH DONT MAKE SENSE!

In [225]:
#convert created_date and deleted_date to datetime.
data[['created_date','deleted_date']] = data[['created_date','deleted_date']].apply(
    lambda _: pd.to_datetime(_))

#create the live_days column.
data['live_days'] = (data['deleted_date'] - data['created_date']).dt.days

C:\Users\Administrator\AppData\Local\Temp\ipykernel_1104\2928096309.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  lambda _: pd.to_datetime(_))
C:\Users\Administrator\AppData\Local\Temp\ipykernel_1104\2928096309.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  lambda _: pd.to_datetime(_))


In [227]:
#drop all cars with first_registration_year of more than 2022
data = data[data['first_registration_year'] <= 2024]

#drop all columns with negative live days. (since you cannot have a listing deleted before it is created)
data = data[data['live_days'] >= 0]

#drop all negative stock days
data = data[data['stock_days'] >= 0]

#drop all cars with price less than 100 euros
data = data[data.price > 100]

In [229]:
columns_to_drop = ['deleted_date', 'created_date', 'first_registration_year', 'article_id', 'ctr']
data = data.drop(columns=columns_to_drop)

In [231]:
data.columns.isnull()

array([False, False, False, False, False, False, False, False])

In [233]:
data.shape

(48808, 8)

In [235]:
from src.eda_first import summarize_dataframe
summarize_dataframe(data)

,Column,Data Type,Unique Values,Missing Values,Sample Unique Values
0,product_tier,object,3,0,"[Basic, Premium, Plus]"
1,make_name,object,81,0,"[Mitsubishi, Ford, Volkswagen, Fiat, Renault]"
2,price,int64,4663,0,"[16750, 1750, 24890, 2850, 23740]"
3,first_zip_digit,int64,9,0,"[5, 6, 7, 2, 3]"
4,search_views,float64,5677,0,"[3091.0, 1856.0, 125.0, 172.0, 1499.0]"
5,detail_views,float64,224,0,"[123.0, 26.0, 6.0, 4.0, 99.0]"
6,stock_days,int64,128,0,"[30, 101, 2, 1, 87]"
7,live_days,int64,345,0,"[31, 101, 2, 87, 62]"


In [237]:
data.describe()

,price,first_zip_digit,search_views,detail_views,stock_days,live_days
count,48808.0000,48808.0000,48808.0000,48808.0000,48808.0000,48808.0000
mean,14917.9609,4.6146,1292.1592,50.1408,35.2497,76.8581
std,15186.5136,2.3615,1351.2076,51.1895,32.5306,82.7472
min,101.0000,1.0000,0.0000,0.0000,0.0000,0.0000
25%,5945.0000,3.0000,327.0000,11.0000,8.0000,15.0000
50%,10950.0000,5.0000,816.0000,31.0000,25.0000,49.0000
75%,18900.0000,7.0000,1796.2500,73.0000,55.0000,110.0000
max,249888.0000,9.0000,7899.0000,223.0000,127.0000,600.0000


In [239]:
data.to_csv('../data/interim/cars_data.csv', index=False)